# Tutorial
目录
* 最简单的常数
* DimFormula对象
* DimVar对象
* 检查公式在量纲上是否正确

引入模块

In [2]:
import Dimanaly as da
import sympy as sp

# 最简单的尝试
<br>我们可以便捷地只用两行代码得到一个无纲量的表达式

In [36]:
AlphaConstant=da.DimFormula(['hbar','epsilon_0','c','e'])#创建量纲公式对象(DimFormula)
AlphaConstant.get_formula()#调用get_formula()方法得到由无量纲量构成的等式

Eq(sqrt(c)*sqrt(epsilon_0)*sqrt(hbar)/e, C)

# DimFormula对象
&emsp;&emsp;创建公式对象(DimFormula)时需要输入物理量的量纲列表,列表中的元素可以是字符或量纲变量(下面会讲到使用DimVar()创建量纲变量对象),字符可以是符号(如上示例)、中文名、英文名或数值等数据库中含有的信息,程序会在数据库中查找该字符的信息并自动把它创建为量纲变量,用户可以选择是否使用正则表达式查找.
<br>&emsp;&emsp;无论是使用正则表达式还是使用非正则表达式,程序只会返回第一个满足条件的变量,第一个指从最左方的列起至最右方的列中的第一个含目标变量的列,然后再提取该列中从上往下第一个满足条件的变量,因为考虑到人们更习惯于使用物理量符号来代表一个物理量,如果使用正则表达式查找的话物理量符号容易与物理量的英文名混淆,因此默认使用非正则表达式也就是使用完全匹配的方式进行查找.

字符可以是中文名、英文名或数值等数据库中含有的信息

In [37]:
AlphaConstant=da.DimFormula(['约化普朗克常数','epsilon_0','c','e'])#创建量纲公式对象(DimFormula)
AlphaConstant.get_formula()#调用get_formula()方法得到由无量纲量构成的等式

Eq(sqrt(c)*sqrt(epsilon_0)*sqrt(hbar)/e, C)

对于不同的自由度,输出的公式形式是不同的,对于自由度为一、二、三及其以上的公式,输出的形式分别如下所示(具体情况可见demonstration.ipynb文件):
$$\frac{\sqrt{l}}{T \sqrt{g}}=\Pi\left(\frac{g l m}{E}\right)$$
$$\Pi\left(\frac{d}{D}, \frac{\rho_{m} v^{2}}{Y}, \frac{1}{\gamma}\right)=0$$
$$\Pi\left(\frac{d h}{\kappa}, \frac{\mu}{d \rho v}, \frac{\kappa}{c \mu}, \frac{v^{2}}{c \theta}, \frac{v}{\kappa l^{2}}, \frac{l}{d}\right)=0$$

可以通过传入关键字参数来设置sympy的符号属性,默认取正,也就是positive_autovar=True

In [38]:
AlphaConstant2=da.DimFormula(['hbar','epsilon_0','c','e'],positive_autovar=False)
lhs=AlphaConstant.get_formula().lhs
lhs2=AlphaConstant2.get_formula().lhs
print('默认为正的情况下对等式左方取对数后展开')
display(sp.expand(sp.log(lhs)))
print('设置为非正的情况下对等式左方取对数后展开')
display(sp.expand(sp.log(lhs2)))

默认为正的情况下对等式左方取对数后展开


log(c)/2 - log(e) + log(epsilon_0)/2 + log(hbar)/2

设置为非正的情况下对等式左方取对数后展开


log(sqrt(c)*sqrt(epsilon_0)*sqrt(hbar)/e)

通过设置regex=True()可以选择使用正则表达式查找,这时只要中英文名、符号等含有给定的字符(不必全匹配)即找到到目标变量

In [3]:
AlphaConstant=da.DimFormula(['hbar','epsilon_0','光.*速','电.*电量'],regex=True)
formula=AlphaConstant.get_formula()
formula

Eq(sqrt(c)*sqrt(epsilon_0)*sqrt(hbar)/e, C)

为求得等式左方的数值,我们可以使用get_formulavalue()方法将有值变量全部替换为给定的数值

In [3]:
AlphaConstant.get_formulavalue()

Eq(3.30262179735227, C)

&emsp;&emsp;在求解数值时也可以灵活地使用Dimanaly模块中的subs_symbol()方法,subs_symbols()方法传入一个符号表达式,将传入的符号表达式中的符号用传入的dimvar变量列表(dimvarlist)中dimvar变量所绑定的数值或sympy符号进行替换,然后返回替换后的结果,用户可以选择将符号替换为绑定的数据或是量纲变量的sympy符号(默认替换数值)
<br>&emsp;&emsp;若想批量替换多个符号表达式,可以使用subs_symbol_list()方法,该方法传入的是符合表达式列表,返回的也是替换后的结果列表,其余都与sub_symbols()方法相同

In [5]:
inverse_alpha=formula.lhs**2*4*sp.pi#得到精细结构常数的倒数表达式
display(1/inverse_alpha)#精细结构常数的表达式
dimvarlist=AlphaConstant.get_dimvarlist()#提取与AlphaConstant对象绑定的dimvarlist属性
result=da.subs_symbol(inverse_alpha,dimvarlist)#使用subs_symbols()方法用数值替换掉其中的符号
result.evalf()#将sympy中的pi取数值

e**2/(4*pi*c*epsilon_0*hbar)

137.065309118907

&emsp;&emsp;若想得到关于某一物理量的表达式可以使用get_formulafor()方法,不过这个方法仅支持一个或两个自由度的情况(对于两个自由度的情况,目标变量应在函数外),该方法基于sympy模块中的solve()方法,考虑到关于某个自变量的解可能不止一个,在这里只返回第一个解,用户可以改变choose_result参数选择返回的解(默认choose_result=0),也可以在使用该方法后从生成的self.solutions属性中查询全部的解

In [42]:
AlphaConstant.get_formulafor('e')

Eq(e, sqrt(c)*sqrt(epsilon_0)*sqrt(hbar)/C)

&emsp;&emsp;创建好DimFormula对象之后,我们可以使用get_dimvardic()得到一个由量纲变量组成的字典,字典的键是量纲变量的符号,字典的值是量纲变量,这样用户可以从量纲变量中得到该变量的符号、数值、量纲等全部信息

In [43]:
AlphaConstant.get_dimvardic()

{'c': <Dimanaly.DimVar at 0x24b33433a90>,
 'hbar': <Dimanaly.DimVar at 0x24b33ddada0>,
 'epsilon_0': <Dimanaly.DimVar at 0x24b339cbf40>,
 'e': <Dimanaly.DimVar at 0x24b31daf250>}

为方便查看量纲变量的量纲矩阵,可以使用get_dimmat()方法,得到一个列表,此列表的最后一行是变量的数值信息,其余行是变量的量纲信息

In [44]:
AlphaConstant.get_dimmat()

,hbar,epsilon_0,c,e
L,2.000000e+00,-3.000000e+00,1.0,0.000000e+00
M,1.000000e+00,-1.000000e+00,0.0,0.000000e+00
T,-1.000000e+00,4.000000e+00,-1.0,1.000000e+00
I,0.000000e+00,2.000000e+00,0.0,1.000000e+00
value_of_SI,1.054561e-34,8.854000e-12,299800000.0,1.602000e-19


# DimVar对象

&emsp;&emsp;对于数据库中没有的变量,用户可以使用DimVar()自定义量纲变量,并传入它的量纲(默认所有量纲都为零)
<br>&emsp;&emsp;在定义的同时可以传入该变量的符号(默认为None)、SI单位制下的值(默认为零)、sympy符号的属性(默认为正,因为这样便于化简)
<br>&emsp;&emsp;在创建量纲变量后可以使用get_dimdic()方法获得量纲字典，使用get_dimseries()方法获得量纲序列，使用get_symbol()方法获得变量的字符符号,使用get_spsymbol()方法获得该变量的sympy符号,get_value_of_SI()获得该量在SI单位制下的值

In [45]:
l=da.DimVar('l',L=1,value_of_SI=1)
print('量纲字典为:')
display(l.get_dimdic())
print('量纲序列为:')
display(l.get_dimseries())
print('字符符号为:')
spsymbol=l.get_symbol()
display(spsymbol)
display('sympy符号为:')
display(l.get_spsymbol())
print('SI单位制下的值为:')
display(l.get_value_of_SI())

量纲字典为:


{'L': 1, 'M': 0.0, 'T': 0.0, 'I': 0.0, 'Theta': 0.0, 'n': 0.0, 'J': 0.0}

量纲序列为:


L        1.0
M        0.0
T        0.0
I        0.0
Theta    0.0
n        0.0
J        0.0
dtype: float64

字符符号为:


'l'

'sympy符号为:'

l

SI单位制下的值为:


1

&emsp;&emsp;若想定义的变量的量纲与数据库中的某个量的量纲相同,可以通过设置use_database_byname参数传入数据库中有关那个变量的信息,这些信息同样也可以是符号、中英文名、数值等数据库中存在的信息,程序会根据这些信息自动在数据库中查找并利用找到的信息自动为量纲变量设置符号、量纲、值等相关信息，若符号、值的信息已经被用户设置好了,那么就会使用用户设置的信息,而未设置的信息仍从数据库中提取

In [46]:
E=da.DimVar('E',use_database_byname='energy',regex=True,positive_autovar=False,complex=False)
E_spsymbol=E.get_spsymbol()
E.get_dimseries()

L        2
M        1
T       -2
I        0
Theta    0
n        0
J        0
dtype: int64

In [47]:
print(E_spsymbol.is_positive)#设置positive_autovar=False后该变量的sympy符号属性就不再为正了
print(E_spsymbol.is_complex)#设置complex=False后该变量的sympy符号属性就不再为复数了

False
False


因为DimVar对象支持正则表达式,所以也可以引入re模块先编译好正则表达式后查找

In [48]:
import re
name=re.compile('HbAr',re.I)#使用re.compile编译正则表达式,修饰符re.I表示使匹配对大小写不敏感
hbar=da.DimVar('hbar',use_database_byname=name,regex=True)
hbar.get_spsymbol()

hbar

<br>&emsp;&emsp;当我们知道某些物理量与其他物理量之间的关系时,可以先创建一个量纲为零(七个量纲指数全为零)的量纲变量,然后采用Dimanaly模块中的change_dim()方法由其他物理量的量纲得到目标物理量的量纲
<br>&emsp;&emsp;change_dim()方法传入的参数是一个字典,字典的键是量纲变量或字符(该字符能在数据库中找到,默认不适用正则表达式查找),字典的值是量纲变量所对应的量纲指数,以构造seebeck系数$\S$、热导率$\kappa$、电导率$\sigma$的量纲为例示范其用法

根据seebeck系数的公式可以得到seebeck系数的量纲与功$W$、电荷量$e$、温度$T$的量纲之间的关系
$$S=\frac{\Delta \phi}{\Delta T}\Rightarrow\left[S\right]=\left[\frac{W}{eT}\right]$$
其中功$W$、电荷量$e$、温度$T$的量纲指数分别为1、-1、-1，下面同理
<br>根据热导率$\kappa$的定义`单位温度梯度单位时间内经单位导热面所传递的热量`可得热导率的量纲表达式
$$\left[\kappa\right]=\left[\frac{W}{t\cdot l^2\cdot T/l}\right]=\left[\frac{W}{tlT}\right]$$
同理根据电导率$\sigma$的定义`单位电势梯度电位时间内经单位面积所传递的电荷量`可得电量的量纲表达式
$$\left[\sigma\right]=\left[\frac{e}{t\cdot l^2\cdot U/l}\right]=\left[\frac{e}{tlU}\right]$$
这样我们可以如下依次得到$s$,$\kappa$,$\sigma$的量纲

In [49]:
S=da.DimVar('s')#初始化seebeck系数量纲,此时它的量纲为0
S.change_dimdic({'W':1,'e':-1,'T':-1})#根据能量、电量、温度的量纲创建seebeck系数的量纲
S.get_dimseries()#得到seebeck系数的量纲

L        2
M        1
T       -3
I       -1
Theta   -1
n        0
J        0
dtype: int64

In [50]:
kappa=da.DimVar('kappa')
kappa.change_dimdic({'W':1,'t':-1,'l':-1,'T':-1})
kappa.get_dimseries()

L        1
M        1
T       -3
I        0
Theta   -1
n        0
J        0
dtype: int64

In [51]:
sigma=da.DimVar('sigma')
sigma.change_dimdic({'e':1,'t':-1,'l':-1,'U':-1})
sigma.get_dimseries()

L       -3
M       -1
T        3
I        2
Theta    0
n        0
J        0
dtype: int64

由上述三个物理量再结合温度$T$可以构造一个关于热电发电效率的无纲量$ZT$值,也即下面输出结果中的等式右方的常数$C$
$$ZT=\frac{S^2\sigma}{\kappa}T$$

In [52]:
Z=da.DimFormula([sigma,S,'T',kappa])
Z.get_formula()

Eq(T*s**2*sigma/kappa, C)

# 检查公式在量纲上是否正确
<br>对于一个公式我们需要将它的项全部移到等式的一边去,然后使用da.is_true()方法判断这些项凑在一起是否为无量纲量,该方法会返回dimseries,dimvarlist并输出对公司正确性的判断结果
$$T=\sqrt{\frac{l}{g}}\Rightarrow C=T^{-1}l^{0.5}g^{-0.5}$$

In [53]:
T=da.DimVar('T',T=1)
dimseries,dimvarlist=da.is_true({T:-1,'l':0.5,'g':-0.5})

the formula is dimensionally correct


因为下式是有纲量,所以判断结果会显示错误,我们可以打印返回的dimseries,dimvarlist来查看这些量凑在一起的量纲以及传入的量纲变量列表
$$\frac{1}{T}\sqrt{lg}\Rightarrow\left[\frac{1}{T}\sqrt{lg}\right]\ne 1\Rightarrow T\ne\sqrt{lg}$$

In [55]:
dimseries,dimvarlist=da.is_true({T:-1,'l':0.5,'g':0.5})
display(dimseries)
display(dimvarlist)

the formula is wrong


L        1.0
M        0.0
T       -2.0
I        0.0
Theta    0.0
n        0.0
J        0.0
dtype: float64